In [1]:
import os
import sys
from pathlib import Path
from datetime import date
from datetime import datetime
import requests
import pandas as pd
import duckdb

In [2]:
from finlab import data
import finlab

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [5]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [6]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 證交所ETF資料讀取  

In [ ]:
# 取得ETF資料
tw_etf_info = requests.get(
    "https://openapi.twse.com.tw/v1/opendata/t187ap47_L", 
    verify=False,
    headers=headers
).json()
print(tw_etf_info)

In [ ]:
# json 轉成 dataframe
tw_etf_info_df = pd.DataFrame(tw_etf_info)
print(tw_etf_info_df.shape)
# tw_etf_info_df.columns

tw_etf_info_df = tw_etf_info_df[["基金代號", "基金名稱", "英文名稱", "標的指數|追蹤指數名稱", "基金類型"]]
tw_etf_info_df["category"] = "ETF"
tw_etf_info_df.columns = ["stock_id", "stock_name", "stock_en_name", "etf_index_name", "etf_type", "industry"]
tw_etf_info_df

In [ ]:
list(tw_etf_info_df.columns)

### 讀取台股企業資訊

In [ ]:
tw_stock_info = data.get('company_basic_info', save_to_storage=True)

print(tw_stock_info.shape)
# tw_stock_info.head(3)

In [ ]:
tw_stock_info.columns

In [ ]:
tw_stock_info = tw_stock_info[["stock_id", "公司簡稱", "英文簡稱", "產業類別"]]
tw_stock_info.columns = ["stock_id", "stock_name", "stock_en_name", "industry"]
tw_stock_info

In [ ]:
tw_stock_info.columns

In [ ]:
tw_etf_info_df.columns

## 合併股票和 ETF 資料

In [ ]:
conn = duckdb.connect(':memory:')

# 直接使用 DuckDB 的 SQL 功能進行合併
combined_df = conn.execute("""
    SELECT 
        stock_id, 
        stock_name, 
        stock_en_name, 
        industry,
        '' as etf_index_name,
        '' as etf_type
    FROM tw_stock_info
    UNION ALL
    SELECT 
        stock_id, 
        stock_name, 
        stock_en_name, 
        industry,
        etf_index_name,
        etf_type
    FROM tw_etf_info_df
""").fetchdf()

print(f"合併完成！總共 {len(combined_df)} 筆資料")
print(f"股票資料: {len(combined_df[combined_df['etf_index_name'] == ''])} 筆")
print(f"ETF資料: {len(combined_df[combined_df['etf_index_name'] != ''])} 筆")
combined_df.head()

## 資料留存ＤＢ

In [16]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [17]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [18]:
table_name = "tw_stock_list"

In [ ]:
# 由於股票清單是完整資料，先刪除現有表格再重新建立
try:
    conn_duckdb.execute(f"DROP TABLE IF EXISTS {table_name}")
    print(f"已刪除現有表格: {table_name}")
except Exception as e:
    print(f"刪除表格時發生錯誤: {e}")

# 將合併後的資料存入資料庫
combined_df.to_sql(table_name, conn_duckdb, if_exists="replace", index=False)
print(f"已將 {len(combined_df)} 筆資料存入資料庫表格: {table_name}")

In [ ]:
# 查詢表中資料來驗證
result_df = conn_duckdb.execute(f"""
    SELECT 
        COUNT(*) as total_count,
        SUM(CASE WHEN etf_index_name = '' THEN 1 ELSE 0 END) as stock_count,
        SUM(CASE WHEN etf_index_name != '' THEN 1 ELSE 0 END) as etf_count
    FROM {table_name}
""").fetch_df()

print("資料庫中的資料統計:")
print(result_df)

print("\n前5筆資料:")
sample_df = conn_duckdb.execute(f"SELECT * FROM {table_name} LIMIT 5").fetch_df()
print(sample_df)

In [ ]:
# 關閉資料庫連線
conn_duckdb.close()
print("資料庫連線已關閉")